Kaggleの仕様上、**`/kaggle/input/` ディレクトリは読み取り専用 (Read-only)** です。

そのため、データセットを作成するフローは以下のようになります。

1.  ノートブック上で特徴量を生成し、**`/kaggle/working/`** (出力ディレクトリ) にFeather形式で保存する。
2.  ノートブックを「Save Version」で実行する。
3.  実行完了後、Outputタブから「New Dataset」を作成する。
4.  次のノートブックでそのDatasetを読み込む（ここで初めて `/kaggle/input/my-new-dataset/` として使えるようになります）。

以下に、**メモリ効率を考慮し、特徴量ごとに個別のFeatherファイルとして保存する** Pythonコードのテンプレートを提案します。

### 特徴量生成・保存用コード

このコードは、TrainとTestを結合して特徴量を作り、元のインデックスに戻してから列ごとにFeatherファイルとして保存する構成になっています。


In [1]:
import os
import glob

files = glob.glob('/kaggle/working/features/*')
for f in files:
    if os.path.isfile(f):
        os.remove(f)
    elif os.path.isdir(f):
        import shutil
        shutil.rmtree(f)


In [2]:
import pandas as pd
import numpy as np
import os
import gc
from pathlib import Path

# ==========================================
# 設定
# ==========================================
COMPETITION_NAME = "competitions/cmi-detect-behavior-with-sensor-data" 
INPUT_PATH = Path(f"/kaggle/input/{COMPETITION_NAME}")

# ルート出力ディレクトリ
OUTPUT_PATH = Path("/kaggle/working/features")

# ==========================================
# ユーティリティ関数
# ==========================================
def load_dataset_with_len(train_file, test_file):
    """
    指定されたファイル名のtrain/testデータを読み込み、
    trainの行数と共に返す
    """
    train_path = INPUT_PATH / train_file
    test_path = INPUT_PATH / test_file
    
    if not train_path.exists() or not test_path.exists():
        print(f"[Warning] {train_file} or {test_file} not found. Skipping...")
        return None, None, 0
    
    print(f"Loading {train_file} and {test_file}...")
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    
    train_len = len(train)
    return train, test, train_len

def save_feature_split(df_feature, feature_name, sub_folder, train_len):
    """
    特徴量を train / test に分割し、それぞれのフォルダ構造に保存する
    
    Args:
        df_feature (pd.DataFrame): 全結合された特徴量データ
        feature_name (str): 特徴量ファイル名
        sub_folder (str): データ由来のフォルダ名 ("main" or "demographics")
        train_len (int): trainデータの行数（分割位置）
    """
    # インデックスをリセット
    df_feature = df_feature.reset_index(drop=True)
    
    # --- Trainデータの保存 ---
    df_train = df_feature.iloc[:train_len].reset_index(drop=True)
    
    # 保存先: features/train/{sub_folder}/
    train_dir = OUTPUT_PATH / "train" / sub_folder
    train_dir.mkdir(parents=True, exist_ok=True)
    
    save_path_train = train_dir / f"{feature_name}.feather"
    df_train.to_feather(save_path_train)
    
    # --- Testデータの保存 ---
    df_test = df_feature.iloc[train_len:].reset_index(drop=True)
    
    # 保存先: features/test/{sub_folder}/
    test_dir = OUTPUT_PATH / "test" / sub_folder
    test_dir.mkdir(parents=True, exist_ok=True)
    
    save_path_test = test_dir / f"{feature_name}.feather"
    df_test.to_feather(save_path_test)
    
    print(f"Saved: {feature_name} (Train: {len(df_train)}, Test: {len(df_test)})")

# ==========================================
# 特徴量生成プロセス
# ==========================================

def process_main_features(full_data, train_len):
    """メインデータの特徴量を作成し、train/testフォルダに分けて保存"""
    print("\n=== Processing Main Data Features ===")
    SUB_FOLDER = "main"
    
    processed_cols = set()

    # 1. ToFセンサー (まとめて保存)
    for i in range(1, 6):
        prefix = f"tof_{i}_"
        target_cols = [c for c in full_data.columns if c.startswith(prefix)]
        
        if len(target_cols) > 0:
            feat_name = f"tof_{i}"
            print(f"Processing {feat_name}...")
            
            df_feat = full_data[target_cols].astype(np.float32)
            
            save_feature_split(df_feat, feat_name, SUB_FOLDER, train_len)
            
            processed_cols.update(target_cols)
            del df_feat
            gc.collect()

    # 2. その他の列 (個別に保存)
    remaining_cols = [c for c in full_data.columns if c not in processed_cols]
    
    for col in remaining_cols:
        series = full_data[col]
        df_feat = pd.DataFrame()
        
        # 型判定
        if pd.api.types.is_numeric_dtype(series):
            # ID系や整数で残すべきもの以外はfloat32化
            if "id" in col and "acc" not in col and "rot" not in col and "thm" not in col:
                 df_feat[col] = series
            else:
                 df_feat[col] = series.astype(np.float32)
        else:
            # カテゴリ化
            df_feat[col] = series.astype('category') # .cat.codes
            
        save_feature_split(df_feat, col, SUB_FOLDER, train_len)
        
        del df_feat, series
        gc.collect()

def process_demographics_features(base_df, train_len):
    """Demographics特徴量を作成し、train/testフォルダに分けて保存"""
    print("\n=== Processing Demographics Data ===")
    SUB_FOLDER = "demographics"
    
    # 1. 読み込み & 結合
    train_demo, test_demo, _ = load_dataset_with_len("train_demographics.csv", "test_demographics.csv")
    if train_demo is None: return

    full_demo = pd.concat([train_demo, test_demo], axis=0, ignore_index=True)
    
    # 2. 集約 (subject単位)
    print("Grouping demographics by subject...")
    demo_grouped = full_demo.groupby('subject').first().reset_index()
    
    del train_demo, test_demo, full_demo
    gc.collect()
    
    # 3. Mainデータの構造に合わせてマージ
    print("Merging demographics onto main data structure...")
    merge_base = base_df[['subject']].copy()
    merged_df = pd.merge(merge_base, demo_grouped, on='subject', how='left')
    
    if 'subject' in merged_df.columns:
        del merged_df['subject']

    # 4. 保存
    for col in merged_df.columns:
        feat_name = f"demo_{col}"
        
        series = merged_df[col]
        df_feat = pd.DataFrame()

        if pd.api.types.is_numeric_dtype(series):
            df_feat[feat_name] = series.astype(np.float32)
        else:
            df_feat[feat_name] = series.astype('category').cat.codes
            
        save_feature_split(df_feat, feat_name, SUB_FOLDER, train_len)
        
        del df_feat
        gc.collect()

    del merged_df, demo_grouped, merge_base
    gc.collect()

def main():
    # 1. メインデータの読み込み (Trainの長さを取得)
    print("Loading Main Data...")
    train, test, train_len = load_dataset_with_len("train.csv", "test.csv")
    
    if train is None: return

    # 全結合
    full_data = pd.concat([train, test], axis=0, ignore_index=True)
    
    del train, test
    gc.collect()

    # 2. Mainデータ処理
    process_main_features(full_data, train_len)
    
    # 3. Demographicsデータ処理
    process_demographics_features(full_data, train_len)

    print("-" * 30)
    print(f"All features saved to {OUTPUT_PATH}")
    print(f"  - Train Main: {OUTPUT_PATH / 'train' / 'main'}")
    print(f"  - Train Demo: {OUTPUT_PATH / 'train' / 'demographics'}")
    print(f"  - Test Main : {OUTPUT_PATH / 'test' / 'main'}")
    print(f"  - Test Demo : {OUTPUT_PATH / 'test' / 'demographics'}")

if __name__ == "__main__":
    main()

Loading Main Data...
Loading train.csv and test.csv...

=== Processing Main Data Features ===
Processing tof_1...
Saved: tof_1 (Train: 574945, Test: 107)
Processing tof_2...
Saved: tof_2 (Train: 574945, Test: 107)
Processing tof_3...
Saved: tof_3 (Train: 574945, Test: 107)
Processing tof_4...
Saved: tof_4 (Train: 574945, Test: 107)
Processing tof_5...
Saved: tof_5 (Train: 574945, Test: 107)
Saved: row_id (Train: 574945, Test: 107)
Saved: sequence_type (Train: 574945, Test: 107)
Saved: sequence_id (Train: 574945, Test: 107)
Saved: sequence_counter (Train: 574945, Test: 107)
Saved: subject (Train: 574945, Test: 107)
Saved: orientation (Train: 574945, Test: 107)
Saved: behavior (Train: 574945, Test: 107)
Saved: phase (Train: 574945, Test: 107)
Saved: gesture (Train: 574945, Test: 107)
Saved: acc_x (Train: 574945, Test: 107)
Saved: acc_y (Train: 574945, Test: 107)
Saved: acc_z (Train: 574945, Test: 107)
Saved: rot_w (Train: 574945, Test: 107)
Saved: rot_x (Train: 574945, Test: 107)
Saved: 

### ポイントと解説

1.  **保存場所 (`OUTPUT_PATH`)**:
    *   `/kaggle/working/features` に保存するようにしています。
    *   実行後、右側のサイドバーの「Output」セクションに `features` フォルダと `.feather` ファイルが表示されます。

2.  **型変換 (`astype(np.float32)`)**:
    *   Kaggleではメモリ制限が厳しいため、デフォルトの `float64` ではなく `float32` にキャストしてから保存することを強く推奨します。ファイルサイズも半分になります。

3.  **Feather形式の利点**:
    *   CSVに比べて読み書きが非常に高速です。
    *   Pandasのデータ型（カテゴリー型など）を保持できます。
    *   **重要**: Featherはindex情報を保持しない（デフォルトのRangeIndexになる）ことが多いため、読み込む際は「行順が変わっていないこと」が前提になります。上記のコードでは `concat` して上から順に保存しているため、読み込み時も同じ順序で読み込めば問題ありません。

4.  **Datasetの作成方法**:
    *   コードを実行し、正常終了したら右上の **[Save Version]** をクリックし、"Save & Run All (Commit)" を選択します。
    *   処理が終わったら、作成されたVersionのViewerページに行き、**[Output]** タブを開きます。